### Task 2 : Prompt Engineering

Testing prompting strategies on the [MTOP Intent Dataset](https://huggingface.co/datasets/mteb/mtop_intent/viewer/de)
> and evaluate the results against each other. The dataset contains instructions and labels indicating on which task the instruction was intended to prompt.

Testing the api 

In [2]:
import os
from openai import OpenAI

client = OpenAI(
    api_key='lm-studio',  
    base_url="http://localhost:1234/v1"
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say only 'This is a test!' ",
        }
    ],
    model="llama-3.2-1b-instruct",
)
print(chat_completion.choices[0].message.content)

This is a test!


Wrapper code from the given file 

In [83]:
def get_answer(prompt, system = None):
    if system is None:
        system = {'role': 'system',
                  'content': 'You are a helpful AI assistant.'}
    chat_completion = client.chat.completions.create(
    messages=[
        system | {
            "role": "user",
            "content": prompt,
        }
    ],
        model="llama-3.2-1b-instruct",
    )
    return chat_completion.choices[0].message.content


def classify_one_example(example, prompt):
    wrapper_prompt = """
    ### Task

    You will be given instructions to a classification task and an input.
    Do answer with only the label you assign based on the instructions provided. 
    The label should only be one word.

    ### Instruction
    {prompt}

    ### Example
    {example}

    ### Label
    """
    return get_answer(wrapper_prompt.format(prompt = prompt, example = example))

Testing on first example

In [77]:
classify_one_example(
    example = 'Today we can go to the playground, since it is sunny outside',
    prompt = ('Classify the sentiment of the given example.' + 
              'Only use 1 for positive, 0 for neutral or -1 for negative as classes.')
)

'0'

Loading Dataset

In [14]:
#loading data from hugging face dataset
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

ds = load_dataset("mteb/mtop_intent", "en")

train_data = ds['train']
val_data = ds['validation']
test_data = ds['test']

#converting in Pandas Dataframe
train_df = train_data.to_pandas()

#checking null values
train_df.isna().sum()


Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 15667
})

In [11]:
#checking different labels 
train_df['label'].unique()

array([  1,  25,  24,   0,  59,  33,  42,  18,  14,  48,  43,  60,  63,
        29,  16,  34,   3,  32,  41,  15,  51,  98,  52,  22,  47,   4,
        11,  49,  38,  71,  57,  86,  53,  55,  44,   7,  36,  97,  50,
        61,  77,   5,  30,  21,  65,  35,  45,  67,  54,  20,  26,  80,
        89,  27,  37,  46,  72,  12,  10,  75,  19,  62,  17,  88, 101,
        69,  31,   6,  68,  56,   9,  40,  92,  85,  66,  74,  13, 110,
         2,  79,  70,  90,  28,  81,  64,  58,   8, 100,  39,  96,  84,
        94, 104, 103, 106,  93,  76,  78, 105,  82,  73, 108, 107,  95,
        91, 102,  23, 112, 109,  83,  99,  87, 111])

In [31]:
# For label = 1 it is a Get Message category (label text value)
train_df_original = train_df.copy()
train_df = train_df[(train_df['label'] == 1) | (train_df['label'] == 0)].set_index('id')

train_df.reset_index(inplace=True)
train_df.head()

,id,text,label,label_text
0,3232343436343136,Has Angelika Kratzer video messaged me?,1,GET_MESSAGE
3,3232353833343937,text Matthew and Helen that are you free,0,SEND_MESSAGE
15,3232353832303532,show me video messages from Atlas,1,GET_MESSAGE
30,3232313731303530,i want you to start recording a video message ...,0,SEND_MESSAGE
71,3232353833373337,I need you to message Zachary Fletcher,0,SEND_MESSAGE
...,...,...,...,...
15599,3231323833303439,Message the list if anyone is going tonight,0,SEND_MESSAGE
15602,3231373233393830,could I video message Brynlee,0,SEND_MESSAGE
15625,3231323831383134,Ask my mom what time I should come over,0,SEND_MESSAGE
15664,3232353735313533,send Izzy that Facebook video message,0,SEND_MESSAGE


First Working with 5 sentences and classifying them 

In [78]:
#fetching only two rows

df = train_df[['text', 'label']]

# Convert DataFrame to a dictionary
data_dict = df.to_dict('records')


Working on 20 examples and testing different prompt strategies 

In [99]:
import pandas as pd
from sklearn.metrics import accuracy_score

prompt_accuracy_list = []

def prompt_strategy(prompt, data, strategy_title):
     
    results = []

    for record in data[0:20]:
        text = record['text']
        true_label = record['label']
        predicted_label = classify_one_example(text,prompt)
        if predicted_label == 'GET_MESSAGE': 
            predicted_label = 1
        elif predicted_label == "OTHER": 
            predicted_label = 0

        # print("predicted_label.........", predicted_label)
        
        if predicted_label == 1 or predicted_label == 0: 
            results.append({
                "text": text,
                "true_label": true_label,
                "predicted_label": predicted_label,
                "prompt_type": 'Prompt 1'
            })

    results_df = pd.DataFrame(results)

    
    accuracy = accuracy_score(results_df['true_label'], results_df['predicted_label'])
    
    
    prompt_accuracy_list.append(pd.DataFrame({
    'Strategy': [strategy_title],
    'Accuracy Score': [accuracy]}))
    
    final_df = pd.concat(prompt_accuracy_list, ignore_index=True)
   
   
    return final_df

In [100]:
prompt_1 = """
Classify on the task instructed by the given example.
Only use 'GET_MESSAGE' or 'OTHER' as class-labels. 
"""

# Processing
prompt_strategy(prompt= prompt_1, data= data_dict, strategy_title="Prompt 1")


,Strategy,Accuracy Score
0,Prompt 1,0.263158


In [101]:
prompt_2 = prompt_1 + """
Think and analyse about which task the instruction prompts before you give the label. 
Take note of verbs in the instruction hinting at the result the user 
is expecting.
"""

prompt_strategy(prompt= prompt_2, data= data_dict, strategy_title="Prompt 2")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053


In [102]:

prompt_3 = prompt_2 + """
Examples should be labeled 'GET_MESSAGE' if the request could be answered by
looking up a message.
In every other case, the label should be 'OTHER'.
"""

prompt_strategy(prompt= prompt_3, data= data_dict, strategy_title="Prompt 3")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222


In [103]:
prompt_4 = prompt_2 + """ These are some examples for class labelling: 
Message the list if anyone is going tonight - OTHER
show me video messages from Atlas - GET_MESSAGE
"""

prompt_strategy(prompt= prompt_4, data= data_dict, strategy_title="Prompt 4")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222
3,Prompt 4,0.250000


In [104]:
prompt_5 = prompt_4 + """Do label the message as a GET_MESSAGE when there is certain kind of a request. For example: 
i want to see texts from cousin Maria- GET_MESSAGE
Did my family text me - GET_MESSAGE
lets see my texts from my wife Brycen - GET_MESSAGE
"""

prompt_strategy(prompt= prompt_5, data= data_dict, strategy_title="Prompt 5")


,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222
3,Prompt 4,0.250000
4,Prompt 5,0.600000


Analysing First 20 Sentences 

In [108]:
for data in data_dict[0:20]: 
    print(data['text'])

Has Angelika Kratzer video messaged me?
text Matthew and Helen that are you free
show me video messages from Atlas
i want you to start recording a video message for Amelia
I need you to message Zachary Fletcher
let's start recording a video message for Sean
Please record a video message for kid
message Angela and tell her that the movie marathon is at Gloria's house in Overland Park
Id like you to send a video
Start recording for a video message to Macie Larita
Message Reggie to ask him what time he will arrive.
MESSAGE DENNIE AND LET HER KNOW I WOULD LIKE TO MEET FOR LUNCH TOMORROW.
please send my family text
Send a happy birthday message to dad.
Send a message to Kevin asking if he has heard from my husband.
Message Morgan's friends that we will have a surprise party for her on September 5th at 8pm
record a video message for Chana Stamps
respond to Kamryn with a video
Has Lila Kiritsy sent me any messages?
text Katey and Helen that are you free


Pre Processing the Text

In [109]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download resources for nltk 
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text, remove_stopwords=True, lemmatize=True):
 
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize the text (split into words)
    words = text.split()

    # Remove stopwords
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

    # Lemmatize words
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

    # Rejoin words into a single string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

# Example Usage
raw_text = "This is an example text, full of unnecessary punctuation!!! And extra spaces... :)"
processed_text = preprocess_text(raw_text)
print("Original Text:", raw_text)
print("Processed Text:", processed_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...


Original Text: This is an example text, full of unnecessary punctuation!!! And extra spaces... :)
Processed Text: example text full unnecessary punctuation extra space


In [112]:
import pandas as pd
from sklearn.metrics import accuracy_score


def prompt_strategy_preprocessed_text(prompt, data, strategy_title):
     
    results = []

    for record in data[0:20]:
        #pre processing the text 
        text = preprocess_text(record['text'])
        true_label = record['label']
        predicted_label = classify_one_example(text,prompt)
        if predicted_label == 'GET_MESSAGE': 
            predicted_label = 1
        elif predicted_label == "OTHER": 
            predicted_label = 0

        
        if predicted_label == 1 or predicted_label == 0: 
            results.append({
                "text": text,
                "true_label": true_label,
                "predicted_label": predicted_label,
                "prompt_type": 'Prompt 1'
            })

    results_df = pd.DataFrame(results)

    
    accuracy = accuracy_score(results_df['true_label'], results_df['predicted_label'])
    
    
    prompt_accuracy_list.append(pd.DataFrame({
    'Strategy': [strategy_title],
    'Accuracy Score': [accuracy]}))
    
    final_df = pd.concat(prompt_accuracy_list, ignore_index=True)
   
   
    return final_df

In [113]:
prompt_strategy_preprocessed_text(prompt= prompt_1, data= data_dict, strategy_title="Prompt 1 Pre-Processed Text")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222
3,Prompt 4,0.250000
4,Prompt 5,0.600000
5,Prompt 1 Pre-Processed Text,0.500000


In [114]:
prompt_strategy_preprocessed_text(prompt= prompt_2, data= data_dict, strategy_title="Prompt 2 Pre-Processed Text")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222
3,Prompt 4,0.250000
4,Prompt 5,0.600000
5,Prompt 1 Pre-Processed Text,0.500000
6,Prompt 2 Pre-Processed Text,0.250000


In [115]:
prompt_strategy_preprocessed_text(prompt= prompt_5, data= data_dict, strategy_title="Prompt 5 Pre-Processed Text")

,Strategy,Accuracy Score
0,Prompt 1,0.263158
1,Prompt 2,0.421053
2,Prompt 3,0.222222
3,Prompt 4,0.250000
4,Prompt 5,0.600000
5,Prompt 1 Pre-Processed Text,0.500000
6,Prompt 2 Pre-Processed Text,0.250000
7,Prompt 5 Pre-Processed Text,0.450000


### Obervation: 

The accuracy is not increasing in every case on Text Pre-Processing